# Best Practices

 Build on the Lambda/kinetis example from module 4
 * add tests to the code (unit tests, integration tests), we will use the library ```pytest``` for that
 * ```pipenv install --dev pytest```, it is only needed for development
 * For working in Visual Studio:
     * Select Python Interpreter > View > Command Pallete > "Select: Python Interpreter"
     * Go in terminal to the current folder and type ```pipenv --venv``` to get the python environment of the virtual environment (```<path>/.local/share/virtualenvs/code-AxO42iuz```
     * Copy the name ```<path>/.local/share/virtualenvs/code-AxO42iuz/bin/python``` to the Command Pallete and choose this interpreter
     * We then get a new icon on the left hand side for testing
     * Add the test path ```<path>/.local/share/virtualenvs/code-AxO42iuz/bin/pytest``` through "configure tests"
 * **Unit tests** only test small units/fractions of the code, **Integration tests**, test the entire code

## Unit Tests
* Create a first test
    * In the test folder, we need to creat a file ```__init__.py```, so that python knows that this is a python package
    * create a file ```model_test.py``` and ```model.py```
    * Test it using docker, we need to add the new created ```model.py``` script to te docker file: ```bash docker build -t stream-model-duration:v2 .```
     
```
docker run -it --rm \
    -p 8080:8080 \
    -e PREDICTIONS_STREAM_NAME="ride_prediction" \
    -e TEST_RUN="True" \
    -e AWS_DEFAULT_REGION="eu-west-1" \
    stream-model-duration:v2
```
    

* To run the tests from the terminal: go to folder ```code```, start virtual env: ```pipenv shell```, then run ```pipenv run pytest tests/``` 

## Integration Test with docker Compose

* We use the file ```test_docker.py``` and add the test at the end:
```
actual_response = response = requests.post(url, json=event)
print('actual_response')
print(json.dumps(actual_response, indent=2)
```
```
expected_response = [{
    'predictions':  [{
        'model': 'ride_duration_prediction_model',
        'version': 'e1efc53e9bd149078b0c12aeaa6365df',#run_id
        'prediction': {
            'ride_duration': 21.294545348333408,
            'ride_id': 256
            }
        }]
    }]

```
* In order to compare the two outcoming dictionaries we use the library ```deepdiff```: ```pipenv install --dev deepdiff```
* To compare floats only up a certain digit we can set a tolerance in deepdiff

* We now use docker-compose instead of docker
    * We create a docker-compose.yaml
    * We create a run.sh script
    * With ```docker-compose up -d``` we can run the container in a detached mode, i.e. we can use the terminal after execution
    * To stop the execution run ```docker-compose down```

## Testing Cloud Services with Localstack

* So far we didn't test the kinesis connection (```class KinesisCallback``` in ```model.py```)
* We will use localstack for that
    * "Fully functional AWS cloud stack"
    * We will use docker-compose to run it and integrate it in our ```docker-compose.yaml``` file
    * To test only the kinesis part we can use ```docker-compose up kinesis```
    * "Use AWS locally": ```aws endpoind-url=http://localhost:4566 kinesis list-streams```
    * Create a stream (locally): ```aws endpoind-url=http://localhost:4566 kinesis create stream --stream-name <value> [--shard-count-value <value>]```, i.e. ```aws endpoind-url=http://localhost:4566 kinesis create stream --stream-name ride-predictions --shard-count-value 1```
    * In our docker-compose.yaml, we specify this by the variable ```KINESIS_ENDPOINT_URL=http://kinesis:4566``` to configure our code to go to localstack istead to aws
    * We also have to add this to our ```model.py``` script. This is done by the function ```create_kinesis_client```


## Code Quality: Linting and Formatting

* PEP8 - Stype Guide for Python
* linters help to see, whether a code follows this guide, e.g. ```pylint```
* "Pylint is a static code analysis tool for the Python programming language."
* ```pipenv install --dev pylint```
* Use this for specific files, e.g. ```pylint model.py``` or to an entire folder ```pylint --recursive=y .``` in the terminal
* Or more convinient in Visual Studio: ```View > Command Palette > Python: Select Linter > pylint```, run it: ```View > Command Palette > run linting```, we see then all suggestions as underlined code
* We can configure what kind of suggestions should be shown. We can create a file ```.pylintrc``` and e.g. add what kind of suggestions should be ignored.
* Alternative to ```pylint```: Many packages (including ```pylint```) use a configuration file called ```pyproject.toml```. Create this file and move the content from ```.pylintrc``` there.
* You can also disable locally some warinings, e.g.: 
```
def lambda_handler(event, context):
    # pylint: disable=unused-argument
    return model_service.lamda_handler(event)
```
* Now we use the packages ```black``` for formatting and ```isort``` for sorting the imports
* ```black --diff . | less```, use ```black --skip-string-normalization --diff . |less``` to ignore single quote issues
* We put this into the ```pyproject.toml``` file
* apply the changes ```black .```
* similar use ```isort --diff . | less```
* We can do this all automatically:
```
isort .
black .
pylint --recursive=y .
pytest tests/
```

## Pre-commit Hooks
* To make sure, the tests we want to do are really done, we can make them always before we commit something to git.
* Use git ```pre-commit hook```: ```pip install pre-commit```
* We use our virtual environment: ```cd code```, ```pipenv shell```, ```pipenv install --dev pre-commit```
* This allows us to define pre-commit hooks
* When we go to the base folder of our repository, we have a folder called ```.git```, in this folder is a folder called ```hooks```. This contains a file called ```pre-commit.sample```
* We only want to run pre-commit hooks for the folder ```code``` in our repo. 
* run ```git init``` in this folder to pretend this is a git repo (delete after the lecture)
* run ```pre-commit```, we need to create a config file ```.pre-commit.yaml```
* create the sample comfig with ```pre-commit sample-config > .pre-commit.yaml```
* ```pre-commit install``` creates a hook at ```.git/hooks/pre-commit```
* When we clone a repo, we need to install the pre-commit hooks!
* When we now commit changes, the pre-commit hooks are run
* We now need to add the hooks we want to run to the ```.pre-commit.yaml``` file

## Makefiles and Make
* create a new file called ```Makefile``` in the ```code``` folder
* when we run ```make run``` this Makefile will be executed.
* Simple example:
```
test:
	echo test
run: test # means that run depends on test
	echo run
```
when we execute ```make run``` we get this output:
```
echo test
test
echo run
run
```